In [1]:
import pandas as pd
import numpy as np
#K-means Function from homework 1
import Kmeans_HW1 as Kmeans

In [2]:
def open_file(txt_file):
    file = pd.read_csv(txt_file, sep='\t', header=-1)
    return file

In [3]:
def main(K):
    '''
    Run the Spectral Clustering method with Kmeans.
    Input: Number of clusters
    Output: accuracy score between the predictions and real values
    '''
    #reading nodes
    nodes = open_file('nodes.txt')
    nodes.set_index(0, inplace=True)
    #reading edges
    edges = open_file('edges.txt')
    edges.sort_values(by=0, inplace=True) 
    #Creating the Adjacency matrix A
    A = pd.crosstab(edges[0], edges[1])
    idx = A.columns.union(A.index)
    A = A.reindex(index = idx, columns=idx, fill_value=0)
    A_indices = A.index
    A = A.values
    #Creating the D matrix
    D = np.diag(np.sum(A, axis=1))
    #Creating the Laplacian matrix
    L = D - A
    #extracting eigenvectors and eigenvalues
    s, v = np.linalg.eigh(L)
    v = v[:, :K]
    #running Kmeans on the eigenvectors
    centers, y_pred, loss_sum = Kmeans.kmeans(v, 2)
    #comparison between real values to predictions
    y_true = nodes.loc[A_indices, 2]
    accuracy_score = np.mean(y_true == y_pred)
    return y_pred, y_true, accuracy_score

In [4]:
#running the process
if __name__ == "__main__":
    y_pred, y_true,score = main(2)
print(score)

0.7369281045751634


In [5]:
score_df = pd.DataFrame({"True labels":y_true, "Predictions":y_pred})
score_df.to_csv("Spectral Clustering comparison.csv")